# Evaluación Docente Institucional

El siguiente código tiene como propósito exponer cómo se recopilan los datos de Evaluación Docente Institucional, su procesamiento a través de python, el cálculo de la participación, transformación de la __escala de percepción__ a la __escala de puntuación__, y el cálculo de los promedios alcanzados por pregunta y competencia.

## Importación de las Librerías

Se recomienda utilizar las siguientes librerías, sin embargo, no es la única forma de realizar este análisis.

In [1]:
# Importación de librerías

import pandas as pd
import numpy as np
import re
import xlsxwriter


## Importación de los Datos

En primera instancia se deberán descargar los archivos disponibles en __Reportes Genéricos__, una plataforma habilitada por informática para el profesional a cargo, deberá descargar todas aquellas que contengan el nombre de estudiantes, se recomienda asignar un nombre diferenciado, debido a que estos se encuentran por Modalidad de formación.

In [69]:
ce1 = pd.read_excel(r'H:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\Estudiantes\CE_1erSemestre.xlsx')
ce2 = pd.read_excel(r'H:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\Estudiantes\CE_2dorSemestre.xlsx')
ptu1 = pd.read_excel(r'H:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\Estudiantes\PTU_1erSemestre.xlsx')
ptu2 = pd.read_excel(r'H:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\Estudiantes\PTU_2doSemestre.xlsx')
tns1 = pd.read_excel(r'H:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\Estudiantes\TNS_1erSemestre.xlsx')
tns2 = pd.read_excel(r'H:\6.-Universidad Arturo Prat - Python\Evaluación Docente 2022\Estudiantes\TNS_2doSemestre.xlsx')

para efectos de este análisis se uso la ruta absoluta del archivo, pero es recomendable utilizar la ruta relativa para futuros trabajos.

## Construcción de Función que limpia los datos

Se crea una función que permite tomar la base de datos en bruto y limpiar los nombres de columnas, titulos, espacios en blancos entre otros aspectos.

In [70]:
def clean_estudiantes(data):
    data = data.iloc[7:]
    data = data.iloc[:, 1:]
    data = data.reset_index(drop=True)
    data.columns = data.iloc[0]
    data = data.iloc[1:]
    data = data.rename(columns=lambda x: re.sub(r'\W+', '_', x.strip().lower()))
    return data

## Aplicación de la Función que limpia los datos

Se aplica la función a las diferentes bases de datos. *Es importante señalar, que el conocimiento de python es muy inicial por ahora, por ende estas funciones pueden mejorar en el futuro.

In [71]:
ce1 = clean_estudiantes(ce1)
ce2 = clean_estudiantes(ce2)
ptu1 = clean_estudiantes(ptu1)
ptu2 = clean_estudiantes(ptu2)
tns1 = clean_estudiantes(tns1)
tns2 = clean_estudiantes(tns2)

## Unión de Bases de datos

In [72]:
Estudiantes2022 = pd.concat([ce1,ce2,ptu1,ptu2,tns1,tns2])

## Seleccionar columnas a eliminar

Debido a las diferentes solicitudes las bases de datos pueden contener entre 1 o 2 columnas más que archivos más antiguos. en este caso se elecciona el sexo del estudiantes porque es irrelevante para este análisis, la edad del docente y el contrato del docente, si bien estos 2 últimos son de importancia para un análisis posterior, estos cambios se aplicaron despues. es recomendable solicitar una base de datos a informática que pueda cruzar los datos, ojalas solicitando identificar el año de nacimiento de la persona con el propósito de calcular la edad actual.

In [73]:
# Columnas a eliminar

eliminar_columnas = ['sexo','edad_docente','contrato_docente']

In [74]:
Estudiantes2022 = Estudiantes2022.drop(eliminar_columnas,axis=1)

## Proceso de Análisis de Participación y Resultados de Evaluación Docente

En el siguiente apartado se generarán dos DataFrames distintos, el primero que medirá los __niveles de participación__ por los estudiantes, y el otro los __resultados por pregunta__. Luego se realizará una unión para poder analizar ambas cosas en un solo documento.

Primero se debe crear objetos que permitan hacer las agrupaciones de manera más sencilla y entendible.

In [75]:
agrupacion_coberturas = ['proceso','semestre','sede','tipo_carrera','unidad_academica','codigo_carrera','nombre_carrera','codigo_actividad','nombre_actividad',
                         'paralelo','tipo_asig','rut_docente','nombre_docente','apaterno_docente','amaterno_docente','estado']

agrupacion_resultados = ['proceso','semestre','sede','tipo_carrera','unidad_academica','codigo_carrera','nombre_carrera','codigo_actividad','nombre_actividad',
                         'paralelo','tipo_asig','rut_docente','nombre_docente','apaterno_docente','amaterno_docente']

estados = ['finalizado','no_iniciada','pendiente']

## Participación

El siguiente análisis busca realizar un conteo de los diferentes estados que puede tener la encuesta de __Evaluación Docente__, los cuales pueden ser:

*finalizado* = Estudiantes participa de manera exitosa en la encuesta
*No iniciada* = Estudiantes no inicia la encuesta, no la contesta, no abre el instrumento
*Pendiente* = Estudiantes por algún motivo, abre el instrumento, pero no termina de contestarlo.

Posterior al conteo de los diferentes estados, se realiza la sumatoria y calculo de la proporción de participación(finalizado).

In [76]:
CoberturasEstudiantes2022 = Estudiantes2022.groupby(agrupacion_coberturas).size().reset_index(name='count')

para dejar los estados anteriores, como columnas independientes se debe *pivotear* la tabla de la siguiente manera:

In [77]:
CoberturasEstudiantes2022 = CoberturasEstudiantes2022.pivot_table(index=agrupacion_resultados, columns='estado',values='count',fill_value=0).reset_index().rename(columns=lambda x: re.sub(r'\W+', '_', x.strip().lower()))

Se generan nuevas columnas, para calcular el total de estudiantes por docente, y la proporcion alcanzada para el estado finalizado:

In [78]:
#Total encuestas

CoberturasEstudiantes2022['total'] = CoberturasEstudiantes2022[estados].sum(axis=1)

In [79]:
CoberturasEstudiantes2022['finalizado_percent']= round((CoberturasEstudiantes2022['finalizado']/CoberturasEstudiantes2022['total'])*100,2)

## Resultados

En el siguiente apartado se calcularán los resultados obtenidos en cada pregunta del instrumento, transformandolo de una *escala de percepción* a una *escala de puntuación* de la siguiente manera:

*Siempre* ~ 3.0
*Generalmente* ~ 2.0
*Pocas veces* ~ 1.0
*Nunca* ~ 0.0
*--* ~ 0.0 = Indicador de no participación

Primero que todo debemos agrupar las columnas que son de respuesta, generar un objeto que contenga la escala de transformación, y generar objetos que contengan los datos de respuesta por cada competencia, y las columnas que se excluirán en el análisis.


In [80]:
# Se agrupan todas las columnas que contienen respuestas

columnas_respuesta = [col for col in Estudiantes2022.columns if col.startswith('respuesta')]

In [81]:
# Se genera la escala de transformación

escala_puntuacion = {'Siempre': 3.0, 'Generalmente': 2.0, 'Pocas veces': 1.0, 'Nunca': 0.0, '--': 0.0}


In [82]:
filtro_finalizado = Estudiantes2022['estado'] == "Finalizado"

In [83]:
# Se generan la agrupación para los datos relacionados con la competencia mediador
respuestas_mediador = [col for col in Estudiantes2022.columns if col.startswith('respuesta_1_')]

In [84]:
# Se genera la agrupación para los datos relacionados con la competencia gestión
respuestas_gestion = [col for col in Estudiantes2022.columns if col.startswith('respuesta_2_')]

In [85]:
columnas_excluidas = ['inicio_aplicacion', 'fin_aplicacion', 'estado', 'fecha_inicio', 'fecha_fin']

Se debe generar el nuevo dataframe que contenga los datos asociados a los resultados.

In [87]:
ResultadosEstudiantes2022 = Estudiantes2022[filtro_finalizado]

In [ ]:
ResultadosEstudiantes2022[columnas_respuesta] = ResultadosEstudiantes2022[columnas_respuesta].replace(escala_puntuacion)

In [90]:
ResultadosEstudiantes2022 = ResultadosEstudiantes2022.drop(columns=columnas_excluidas)

In [91]:
ResultadosEstudiantes2022 = ResultadosEstudiantes2022.groupby(agrupacion_resultados).mean(columnas_respuesta).round(2).reset_index()

## Union de Bases de datos

A continuación a traves de *merge* se unirán los objetos __CoberturasEstudiantes2022__ y __ResultadosEstudiantes2022__, de los cuales se creará un nuevo objeto que contenga ambos datos.

In [92]:
data = CoberturasEstudiantes2022.merge(
    right = ResultadosEstudiantes2022,
    how = "outer",
    on = agrupacion_resultados
)

## Cálculo de nuevas columnas

Luego de unificar las bases de datos, se crearan nuevas variables como la puntuación promedio alcanzada por competencia:

*Medidador* = el promedio de la sumatoria de los promedios alcanzados por pregunta, de la 1 a la 13
*Gestión* = el promedio de la sumatoria de los promedios alcanzados por pregunta de la 14 a la 15


In [95]:
data[columnas_respuesta]= data[columnas_respuesta].fillna(0)

In [97]:
data['mediador'] = data[respuestas_mediador].mean(axis=1).round(2)

In [98]:
data['gestion'] = data[respuestas_gestion].mean(axis=1).round(2)

In [99]:
writer = pd.ExcelWriter('estudiantes2022.xlsx', engine='xlsxwriter')
data.to_excel(writer, sheet_name='estudiantes2022', index=True)
writer.save()

C:\Users\aigna\AppData\Local\Temp\ipykernel_9996\218577113.py:3: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
